### 1. Tạo pertured data và lưu giống data cũ (cal_metric.py)
    1. Tạo ra câu pertured va convert json và ghép label (pertured_folder/masked_data_json)
### 2. Lấy ra vector logit 
    1. convert interim -> txt 
    2. transform, prepare -> json 
    3. đưa json (masked data 0.1) vào model lấy logit, prediction -> cần tạo class để lấy cho cả origin data và masked data
### 3. convert label gold theo từ thành label gold theo token
### 4. Tính inf 

## Step 2

In [ ]:
# 2.1 Done
# convert csv to coNLL format and write to txt file
!python ../data_transformations.py --transform_file '/mnt/c/Users/Phat Pham/Documents/THESIS/SRLPredictionEasel/SRL/transform_file_mlm.yml'
# output file MLM/coNLL_txt

In [ ]:
# 2.2 Done
!python ../data_transformations.py --transform_file '/mnt/c/Users/Phat Pham/Documents/THESIS/SRLPredictionEasel/SRL/transform_file_conll.yml'
# nhớ chỉnh lại đường dẫn trong file transform_file_conll.yml

In [ ]:
# 2.2 Done
# !python ../data_preparation.py --task_file tasks_file_SRL.yml --data_dir ../data/coNLL_tsv --max_seq_len 50

!python ../data_preparation.py \
        --task_file "/mnt/c/Users/Phat Pham/Documents/THESIS/SRLPredictionEasel/SRL/tasks_file_SRL.yml" \
        --data_dir ../MLM/coNLL_tsv_json \
        --max_seq_len 85

## Calculation influence, relevance and compentence score

In [ ]:

!python metrics_calculation.py \
    --data_mask_dir './data_mlm/perturbed_data/avg_neg_cos/' \
    --data_origin_dir './data_mlm/process_folder/coNLL_tsv_json/ner_json/'  \
    --model_path '../output/multi_task_model_9_13050.pt' \
    --log_name "comp_avg_neg_cos" 


In [26]:
!python metrics_calculation.py \
    --data_mask_dir './data_mlm/process_folder/mlm_output/' \
    --data_origin_dir './data_mlm/process_folder/coNLL_tsv_json/ner_json/'  \
    --model_path '../output/multi_task_model_9_13050.pt' \
    --log_name "comp_del_mask_token" \
    --del_mask_token True

2024-05-31 15:13:12.148376: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-31 15:13:12.294042: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-31 15:13:13.400666: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/phatpham/anaconda3/envs/min_ds-env/lib/python3.8/site-packages/torch/cuda/__init__.py:546: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2024-05-31 15:13:16.173330: E tensorflow/compiler/xla/stream_executor/cuda/cu